In [6]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from scipy import sparse

#设置要处理的stack
model_type = 'gbdt' # 模型类型
model_num = 5 # 序列号

In [8]:
# 加载每一个fold的基模型
def load_stack_models(model, num, name1, name2, name3, name4):
    model1 = lgb.Booster(model_file='result/stack_fusion/{}/stack{}/{}'.format(model, num, name1))
    model2 = lgb.Booster(model_file='result/stack_fusion/{}/stack{}/{}'.format(model, num, name2))
    model3 = lgb.Booster(model_file='result/stack_fusion/{}/stack{}/{}'.format(model, num, name3))
    model4 = lgb.Booster(model_file='result/stack_fusion/{}/stack{}/{}'.format(model, num, name4))
    return model1, model2, model3, model4

    
model1, model2, model3, model4 = load_stack_models(model_type, model_num, 
                                                   '234_ml126_mcw1_sa0.7_cs0.7_5000_697148____base_ctr_1_17', 
                                                   '134_ml126_mcw1_sa0.7_cs0.7_5000_694712____base_ctr_1_12', 
                                                   '124_ml800_mcw1_sa0.7_cs0.7_5000_699522____base_ctr_0_50', 
                                                   '123_ml800_mcw1_sa0.7_cs0.7_5000_698721____base_ctr_0_47')

In [4]:
test2 = sparse.load_npz('data/test/test2_9_2162.npz')

In [5]:
res2 = pd.read_csv('data/test/res2.csv')

In [6]:
test2.shape

(11727304, 2162)

In [7]:
res2.shape

(11727304, 2)

In [10]:
print('reading test2...')
test_x = test2
print('test shape:', test_x.shape)
print('predicting...')

for i,model in enumerate([model1, model2, model3, model4],start=1):
    res2['score'] = model.predict(test_x)
    res2['score'] = res2['score'].apply(lambda x: float('%.6f' % x))
    
    print('saving submission to result/test{}_predict{}.csv'.format(2, i))
    res2.to_csv('result/test{}_predict{}.csv'.format(2, i), index=False)


reading test2...
test shape: (11727304, 2162)
predicting...
saving submission to result/test2_predict1.csv
saving submission to result/test2_predict2.csv
saving submission to result/test2_predict3.csv
saving submission to result/test2_predict4.csv


In [10]:
# 加载训练集
data1 = sparse.load_npz('data/tr1/tr1_9_2162.npz')
data2 = sparse.load_npz('data/tr2/tr2_9_2162.npz')
data3 = sparse.load_npz('data/tr3/tr3_9_2162.npz')
data4 = sparse.load_npz('data/tr4/tr4_9_2162.npz')

In [11]:
# 在各训练集上预测
pre1 = model1.predict(data1)
print('pre1 compelted!')
del data1
pre2 = model2.predict(data2)
print('pre2 compelted!')
del data2
pre3 = model3.predict(data3)
print('pre3 compelted!')
del data3
pre4 = model4.predict(data4)
print('pre4 compelted!')
del data4

pre1 compelted!
pre2 compelted!
pre3 compelted!
pre4 compelted!


In [12]:
# 保存预测结果，作为模型融合阶段的特征
predict1 = pd.DataFrame({'prob':pre1})
predict2 = pd.DataFrame({'prob':pre2})
predict3 = pd.DataFrame({'prob':pre3})
predict4 = pd.DataFrame({'prob':pre4})
predict1.to_csv('result/stack_fusion/{}/stack{}/tr1_predict.csv'.format(model_type, model_num),index=False)
predict2.to_csv('result/stack_fusion/{}/stack{}/tr2_predict.csv'.format(model_type, model_num),index=False)
predict3.to_csv('result/stack_fusion/{}/stack{}/tr3_predict.csv'.format(model_type, model_num),index=False)
predict4.to_csv('result/stack_fusion/{}/stack{}/tr4_predict.csv'.format(model_type, model_num),index=False)